In [2]:
# Import usual Python and data handling stuff
import numpy as np
import pandas as pd

In [3]:
# Not yet used - No graph :-(
#import matplotlib.pyplot as plt
#%matplotlib inline

# Analysis of the participations in EU Framework Programmes

In the analysis, we cover the participation in the EU Framework Programmes for Research and Innovation between 2007 and 2017. This period is covered by FP7 (2007-2013) and Horizon 2020 (since 2014).

## FP7 datasets: 2007-2013

### Set up general coding information from Cordis

Cordis has several dictionaries for coded information. The following datasets are created on this basis and will be used to "decode" the information provided in the projects and organisations datasets for FP7.

In [4]:
# Symbols and names of countries in several langues
country_file = "FP7/cordisref-countries.xls"
countries = pd.read_excel(
    country_file,
    sheet_name="cordisref-countries",
    header=0,
)

In [5]:
# Symbols and names of FP7 sub-programmes in several langues
fp7_programmes_file = "FP7/cordisref-FP7programmes.xls"
fp7_programmes = pd.read_excel(
    fp7_programmes_file,
    sheet_name="Hoja1",
    header=0,
)

In [6]:
# Full names of funding schemes
fp7_schemes_file = "FP7/cordisref-projectFundingSchemeCategory.xls"
fp7_schemes = pd.read_excel(
    fp7_schemes_file,
    sheet_name="cordisref-projectFundingSchemeC",
    header=0,
)

### FP7 projects and organisations
The data for projects and participations in FP7 are available on the Open Data Portal of the European Union:

https://data.europa.eu/euodp/en/data/dataset/cordisfp7projects

Two datasets will be created:
* `fp7_proj`: with the descriptors of the FP7 funded projects
* `fp7_part`: with the descriptions of the participating organisations

These two datasets will be merged into one sigle `fp7` dataset where the project informations will be repeated to each participations.

In [7]:
%%time

fp7_proj_file = "FP7/cordis-fp7projects.xlsx"
fp7_part_file = "FP7/cordis-fp7organizations.xlsx"

fp7_proj = pd.read_excel(
    fp7_proj_file, sheet_name="cordis-fp7projects",
    header=0,
)
fp7_part = pd.read_excel(
    fp7_part_file,
    sheet_name="cordis-h2020organizations", # Which is a mistake in the Cordis dataset!
    header=0,
)

CPU times: user 21.9 s, sys: 104 ms, total: 22 s
Wall time: 22 s


In [8]:
# We  rename some columns which are in both datasets:
# In projects:
fp7_proj = fp7_proj.rename(
    index=str,
    columns={
        "id": "projectID",
        "rcn": "projectRCN",
        "acronym": "projectAcronym"
    }
)
# In participations:
fp7_part = fp7_part.rename(
    index=str,
    columns={
        "id": "organizationID",
        "projectRcn": "projectRCN",
        "name": "organizationName",
    }
)

Some project acronyms are not consistent in their writing (lower case), so we force upper case writing:

In [9]:
fp7_proj["projectAcronym"] = fp7_proj["projectAcronym"].str.upper()
fp7_part["projectAcronym"] = fp7_part["projectAcronym"].str.upper()

So, let's merge these two datasets and create the `fp7` dataset.

In [10]:
fp7 = fp7_proj.merge(
    fp7_part,
    on=["projectRCN","projectID","projectAcronym"]
)

In [11]:
fp7_proj.shape, fp7_part.shape, fp7.shape

((25778, 21), (146021, 23), (146021, 41))

## Horizon 2020 datasets: 2014-2020

### Set up general coding information from Cordis

Cordis has several dictionaries for coded information. The following datasets are created on this basis and will be used to "decode" the information provided in the projects and organisations datasets for FP7 and Horizon 2020.

In [12]:
# Symbols and names of Horizon 2020 sub-programmes in several languages
h2020_programmes_file = "Horizon 2020/cordisref-H2020programmes.xls"
h2020_programmes = pd.read_excel(
    h2020_programmes_file,
    sheet_name="Hoja1",
    header=0,
)

In [13]:
# Symbols and names of Horizon 2020 topics in several languages
h2020_topics_file = "Horizon 2020/cordisref-H2020topics.xlsx"
h2020_topics = pd.read_excel(
    h2020_topics_file,
    sheet_name="cordisref-H2020topics",
    header=0,
)

In [14]:
# Symbols and names of Horizon 2020 research topics
h2020_sic_file = "Horizon 2020/cordisref-sicCode.xls"
h2020_sic = pd.read_excel(
    h2020_sic_file,
    sheet_name="cordisref-sicCode",
    header=0,
)

### Horizon 2020 projects and organisations
The data for projects and participations in Horizon 2020 are available on the Open Data Portal of the European Union:

https://data.europa.eu/euodp/en/data/dataset/cordisH2020projects

The cut-off date is: 2017-10-12

Two datasets will be created:
* `h2020_proj`: with the descriptors of the Horizon 2020 funded projects
* `h2020_part`: with the descriptions of the participating organisations

These two datasets will be merged into one sigle `h2020` dataset where the project informations will be repeated to each participations.


In [15]:
%%time

h2020_proj_file = "Horizon 2020/cordis-h2020projects.csv"
h2020_part_file = "Horizon 2020/cordis-h2020organizations.xlsx"

h2020_proj = pd.read_csv(
    h2020_proj_file,
    sep=";",
    header=0,
)
h2020_part = pd.read_excel(
    h2020_part_file,
    sheet_name="organisation",
    header=0,
)

CPU times: user 6.42 s, sys: 28 ms, total: 6.45 s
Wall time: 6.45 s


In [16]:
# We  rename some columns which are in both datasets:
# In projects:
h2020_proj = h2020_proj.rename(
    index=str,
    columns={
        "id": "projectID",
        "rcn": "projectRCN",
        "acronym": "projectAcronym"
    }
)
# In participations:
h2020_part = h2020_part.rename(
    index=str,
    columns={
        "id": "organizationID",
        "projectRcn": "projectRCN",
        "name": "organizationName",
    }
)

Two projects with projectRCN have non-consistent project acronyms, so let's force them to the same writing:
* projectRCN = 208306
* projectRCN = 194607

In [17]:
h2020_part.loc[h2020_part.projectRCN.eq(208306)] =\
    h2020_proj.loc[h2020_proj.projectRCN.eq(208306)].projectAcronym[0]
h2020_part.loc[h2020_part.projectRCN.eq(194607)] =\
    h2020_proj.loc[h2020_proj.projectRCN.eq(194607)].projectAcronym[0]

So, let's merge these two datasets and create the `h2020` dataset.

In [18]:
h2020 = h2020_proj.merge(
    h2020_part,
    on=["projectRCN", "projectID", "projectAcronym"]
)

In [19]:
h2020_proj.shape, h2020_part.shape, h2020.shape

((14837, 21), (71312, 23), (62495, 41))

We can see from the fusion that not every project record number has found its corresponding record number in the other dataset. We find indeed 8806 project record numbers (projectRCN) in the `h2020_part` dataset that are _not_ matched in the `h2020_proj` dataset, which is exactly the difference between the size of `h2020_part` and `h2020`.

In [20]:
rcn_in_proj = h2020_proj["projectRCN"].unique()
missing_rcn = h2020_part.loc[~h2020_part["projectRCN"].isin(rcn_in_proj),"projectRCN"]

In [21]:
len(missing_rcn) + h2020.shape[0] == h2020_part.shape[0]

True

## Analysis

This parts is the generala analysis of the FP7 participations dataset extracted from the Cordis project files (`fp7` and `h2020` datasets).

We list below all the column headers of the datasets.

In [22]:
h2020.columns.difference(fp7.columns)

Index([], dtype='object')

So we see that both datasets, `fp7` and `h2020`have the same set of columns.